<a href="https://colab.research.google.com/github/leticiarccorrea/sales-operations-demand-forecasting/blob/main/case_salesmodel_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import dataset

from google.colab import drive
import pandas as pd


# Access to Google Drive
drive.mount('/content/drive')
datapah = '/content/drive/MyDrive/caseboti/dataset.csv'
datapah_dic = '/content/drive/MyDrive/caseboti/dicionariodedados.csv'


# Load file in pandas and spark
base = pd.read_csv(datapah, sep=';', on_bad_lines='warn')
base_dictionary = pd.read_csv(datapah_dic)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import bibliotecas

import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.metrics import mean_absolute_error



In [3]:
# import de bibliotecas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error


from typing import Dict

# 1. Ajuste e limpeza dataset

In [4]:
# criando uma base para não alterar a original
base_work = base.copy()

# converter colunas numéricas

# Define colunas numéricas
numeric_like_columns = [
    "QT_VENDA_BRUTO",
    "QT_DEVOLUCAO",
    "VL_RECEITA_BRUTA",
    "VL_RECEITA_LIQUIDA",
    "PCT_DESCONTO",
    "VL_PRECO",
]

def parse_ptbr_number(series: pd.Series) -> pd.Series:
    if pd.api.types.is_numeric_dtype(series):
        return series

    # Normaliza tokens ausentes e converte PT-BR - float
    return (
        series.astype(str)
        .str.strip()
        .replace(
            {
                "": np.nan,
                "nan": np.nan,
                "NaN": np.nan,
                "NULL": np.nan,
                "None": np.nan,
            }
        )
        .str.replace(".", "", regex=False)   # remove separador de milhar
        .str.replace(",", ".", regex=False)  # vírgula decimal -> ponto
        .astype(float)
    )

# aplicando na base
for col in numeric_like_columns:
    if col in base_work.columns:
        base_work[col] = parse_ptbr_number(base_work[col])

# Preencher desconto faltante com 0 (assimundo que não teve desconto)
if "PCT_DESCONTO" in base_work.columns:
    base_work["PCT_DESCONTO"] = base_work["PCT_DESCONTO"].fillna(0.0)

    # normalizar desconto para 0–1
    if base_work["PCT_DESCONTO"].max() > 1.0:
        base_work["PCT_DESCONTO"] = base_work["PCT_DESCONTO"] / 100.0

    # Clip de ajuste nos descontos negativos
    base_work["PCT_DESCONTO"] = base_work["PCT_DESCONTO"].clip(lower=0.0)

# Criar uma coluna de demanda líquida: qt vendida - qt devolvida
if "QT_DEVOLUCAO" in base_work.columns:
    base_work["QT_DEVOLUCAO"] = base_work["QT_DEVOLUCAO"].fillna(0.0)
else:
    base_work["QT_DEVOLUCAO"] = 0.0

base_work["qt_venda_liquida"] = base_work["QT_VENDA_BRUTO"] - base_work["QT_DEVOLUCAO"]

# % devolução
base_work["pct_devolucao"] = np.where(
    base_work["QT_VENDA_BRUTO"] > 0,
    base_work["QT_DEVOLUCAO"] / base_work["QT_VENDA_BRUTO"],
    np.nan,
)

# Features de tempo baseada no COD_CICLO (formato: YYYYCC)
base_work["ano"] = (base_work["COD_CICLO"] // 100).astype(int)
base_work["ciclo"] = (base_work["COD_CICLO"] % 100).astype(int)

# Criar colune de flag resumo de campanha
campaign_columns = [c for c in base_work.columns if c.startswith("FLG_CAMPANHA")]

# se não encontrar colunas, cria como 0
if len(campaign_columns) > 0:
    base_work["FLG_CAMPANHA_RESUMO"] = (
        base_work[campaign_columns]
        .fillna(0)
        .max(axis=1)
        .astype(int)
    )
else:
    base_work["FLG_CAMPANHA_RESUMO"] = 0



In [5]:
# Métricas

def wape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    WAPE = sum(|erro|) / sum(|real|).
    - Métrica para previsão de demanda.
    - Penaliza erros proporcionalmente ao volume.
    """
    denom = np.sum(np.abs(y_true)) + 1e-6
    return float(np.sum(np.abs(y_true - y_pred)) / denom)

def bias(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Bias = sum(pred - real) / sum(real).
    - Ajuda a detectar tendência de superprevisão vs subprevisão.
    """
    denom = np.sum(y_true) + 1e-6
    return float(np.sum(y_pred - y_true) / denom)

def smape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    sMAPE = média do erro percentual simétrico.
    - usada quando há muitos valores baixos/zeros.
    """
    denom = (np.abs(y_true) + np.abs(y_pred)) + 1e-6
    return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom))

def summarize_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    return {
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "wape": wape(y_true, y_pred),
        "smape": smape(y_true, y_pred),
        "bias": bias(y_true, y_pred),
    }


In [6]:
# Limpeza e padronização do dataset

def clean_base_work(base_work: pd.DataFrame) -> pd.DataFrame:
    """
    Limpa e padroniza a base para reduzir risco de erro silencioso em modelagem.

    Regras aplicadas:
    - Tipos numéricos e categóricos coerentes
    - Clip de desconto em [0,1]
    - Demanda líquida negativa truncada em 0 (estabilidade + semântica)
    - Remoção de linhas sem COD_CICLO ou sem target
    """

    # Checagens mínimas
    required_cols = [
        "COD_CICLO",
        "qt_venda_liquida",
        "PCT_DESCONTO",
        "VL_PRECO",
        "FLG_CAMPANHA_RESUMO",
        "COD_MATERIAL",
        "COD_CANAL",
        "COD_REGIAO",
    ]
    missing_cols = [c for c in required_cols if c not in base_work.columns] # Changed df.columns to base_work.columns
    if missing_cols:
        raise ValueError(f"Colunas obrigatórias ausentes no base_work: {missing_cols}")

    # Tipos principais
    base_work["COD_CICLO"] = pd.to_numeric(base_work["COD_CICLO"], errors="coerce") # Added missing '['

    for col in ["qt_venda_liquida", "PCT_DESCONTO", "VL_PRECO", "pct_devolucao"]:
        if col in base_work.columns: # Changed df.columns to base_work.columns
            base_work[col] = pd.to_numeric(base_work[col], errors="coerce") # Added missing '[' for base_work[col]

    base_work["FLG_CAMPANHA_RESUMO"] = pd.to_numeric(base_work["FLG_CAMPANHA_RESUMO"], errors="coerce").fillna(0).astype(int) # Changed df to base_work

    # Categóricas principais (evitar virar numérico e "criar ordem" artificial)
    base_work["COD_MATERIAL"] = base_work["COD_MATERIAL"].astype(str)
    base_work["COD_CANAL"] = base_work["COD_CANAL"].astype(str)
    base_work["COD_REGIAO"] = base_work["COD_REGIAO"].astype(str)

    # Limpeza de NaNs essenciais
    base_work = base_work.dropna(subset=["COD_CICLO", "qt_venda_liquida"]).copy()

    # Regra de demanda líquida
    base_work["qt_venda_liquida"] = np.clip(base_work["qt_venda_liquida"], a_min=0, a_max=None)

    # Regra de desconto
    base_work["PCT_DESCONTO"] = base_work["PCT_DESCONTO"].fillna(0.0).clip(lower=0.0)

    # Preço: remover infinitos
    base_work["VL_PRECO"] = base_work["VL_PRECO"].replace([np.inf, -np.inf], np.nan) # Changed df to base_work

    # Features de tempo
    if "ano" not in base_work.columns: # Changed df.columns to base_work.columns
        base_work["ano"] = (base_work["COD_CICLO"] // 100).astype(int) # Changed df to base_work
    if "ciclo" not in base_work.columns: # Changed df.columns to base_work.columns
        base_work["ciclo"] = (base_work["COD_CICLO"] % 100).astype(int) # Changed df to base_work

    return base_work.reset_index(drop=True) # Changed df to base_work

base_clean = clean_base_work(base_work)
base_clean.shape

(173923, 23)

In [7]:
base_clean.head()

,COD_CICLO,FLG_DATA,COD_MATERIAL,COD_CANAL,DES_CATEGORIA_MATERIAL,DES_MARCA_MATERIAL,COD_REGIAO,QT_VENDA_BRUTO,QT_DEVOLUCAO,VL_RECEITA_BRUTA,...,FLG_CAMPANHA_MKT_C,FLG_CAMPANHA_MKT_D,FLG_CAMPANHA_MKT_E,PCT_DESCONTO,VL_PRECO,qt_venda_liquida,pct_devolucao,ano,ciclo,FLG_CAMPANHA_RESUMO
0,201917,1,431148,anon_S0,anon_S2,anon_S3,anon_S1,11934.0,414.0,431869.08,...,0,0,0,0.00,455.4,11520.0,0.034691,2019,17,0
1,202005,0,177816,anon_S0,anon_S2,anon_S4,anon_S1,540.0,252.0,27743.40,...,0,0,0,0.00,773.4,288.0,0.466667,2020,5,0
2,201901,0,171786,anon_S0,anon_S5,anon_S6,anon_S1,54012.0,1410.0,962860.20,...,0,0,0,0.35,341.4,52602.0,0.026105,2019,1,1
3,201813,0,177774,anon_S7,anon_S2,anon_S8,anon_S1,438.0,0.0,7608.60,...,0,0,0,0.00,450.9,438.0,0.000000,2018,13,0
4,202006,1,446592,anon_S0,anon_S5,anon_S9,anon_S1,2760.0,240.0,83339.40,...,0,0,0,0.00,431.4,2520.0,0.086957,2020,6,0


In [12]:
# Engenharia de variáveis
#    - Lags e rollings por série (material/canal/região)
#    - Features táticas (campanha, desconto, preço)

def make_discount_bucket(discount: float) -> str:
    """Bucket de desconto para feature categórica e leitura de negócio."""
    if pd.isna(discount) or discount <= 0:
        return "0"
    if discount <= 0.10:
        return "0-10%"
    if discount <= 0.20:
        return "10-20%"
    if discount <= 0.30:
        return "20-30%"
    if discount <= 0.40:
        return "30-40%"
    return "40%+"


def aggregate_for_forecast(
    df: pd.DataFrame,
    group_keys: List[str],
    target_col: str = "qt_venda_liquida",
) -> pd.DataFrame:
    """
    Agrega a base para a granularidade de modelagem.

    - target: soma (demanda total do grupo no ciclo)
    - desconto/preço: média (proxy tática)
    - campanha/evento: max (se houve)
    """
    agg_dict = {
        target_col: "sum",
        "PCT_DESCONTO": "mean",
        "VL_PRECO": "mean",
        "FLG_CAMPANHA_RESUMO": "max",
    }

    if "pct_devolucao" in df.columns:
        agg_dict["pct_devolucao"] = "mean"

    # Opcional: manter descritores do material (se existirem e forem estáveis)
    for c in ["DES_CATEGORIA_MATERIAL", "DES_MARCA_MATERIAL"]:
        if c in df.columns:
            agg_dict[c] = "first"

    # Evento (se existir)
    if "FLG_DATA" in df.columns:
        agg_dict["FLG_DATA"] = "max"

    base_agg = (
        df.groupby(group_keys, as_index=False)
          .agg(agg_dict)
          .sort_values(group_keys)
          .reset_index(drop=True)
    )

    # Recriar ano/ciclo a partir do COD_CICLO (garante consistência)
    if "COD_CICLO" in base_agg.columns:
        base_agg["ano"] = (base_agg["COD_CICLO"] // 100).astype(int)
        base_agg["ciclo"] = (base_agg["COD_CICLO"] % 100).astype(int)

    return base_agg

group_keys_agg = ["COD_CICLO", "COD_MATERIAL", "COD_CANAL", "COD_REGIAO"]
base_agg = aggregate_for_forecast(base_clean, group_keys=group_keys_agg)

def add_time_series_features(
    df: pd.DataFrame,
    series_keys: List[str],
    time_col: str = "COD_CICLO",
    target_col: str = "qt_venda_liquida",
    lag_list: Optional[List[int]] = None,
) -> pd.DataFrame:
    """
    Cria features temporais robustas:
    - lags (1,2,3,6,12)
    - roll mean/std/cv com shift(1) (sem leak)
    - interações de campanha com desconto
    """
    if lag_list is None:
        lag_list = [1, 2, 3, 6, 12]

    out = df.copy()
    out = out.sort_values(series_keys + [time_col]).reset_index(drop=True)

    # Features táticas
    out["has_discount"] = (out["PCT_DESCONTO"] > 0).astype(int)
    out["discount_bucket"] = out["PCT_DESCONTO"].apply(make_discount_bucket)
    out["log_price"] = np.log1p(np.clip(out["VL_PRECO"].fillna(0), a_min=0, a_max=None))
    out["campaign_discount_intensity"] = out["FLG_CAMPANHA_RESUMO"] * out["PCT_DESCONTO"]

    # Lags
    for l in lag_list:
        out[f"y_lag_{l}"] = out.groupby(series_keys)[target_col].shift(l)
        out[f"discount_lag_{l}"] = out.groupby(series_keys)["PCT_DESCONTO"].shift(l)
        out[f"price_lag_{l}"] = out.groupby(series_keys)["VL_PRECO"].shift(l)

    # Rolling sem vazamento: sempre usar shift(1)
    y_shift_1 = out.groupby(series_keys)[target_col].shift(1)

    # Criar id de grupo para rolling por série
    group_id = out[series_keys].astype(str).agg("|".join, axis=1)

    out["y_roll_mean_3"] = (
        y_shift_1.groupby(group_id).rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)
    )
    out["y_roll_mean_6"] = (
        y_shift_1.groupby(group_id).rolling(6, min_periods=1).mean().reset_index(level=0, drop=True)
    )
    out["y_roll_std_6"] = (
        y_shift_1.groupby(group_id).rolling(6, min_periods=2).std().reset_index(level=0, drop=True)
    )
    out["y_roll_cv_6"] = out["y_roll_std_6"] / out["y_roll_mean_6"].replace(0, np.nan)

    # Demanda relativa vs baseline local (ajuda a capturar picos)
    out["y_rel_vs_roll_6"] = out[target_col] / out["y_roll_mean_6"].replace(0, np.nan)

    # Ciclo do ano como categórico "sazonal"
    out["ciclo_str"] = out["ciclo"].astype(str).str.zfill(2)

    return out

time_series_group_cols  = ["COD_MATERIAL", "COD_CANAL", "COD_REGIAO"]
base_clean_model = add_time_series_features(base_agg, series_keys=time_series_group_cols)

# Remover início de série (sem lag 1) para evitar treinar sem histórico
base_clean_model = base_clean_model.dropna(subset=["y_lag_1"]).reset_index(drop=True)
base_clean_model.shape




(166810, 39)

In [13]:
base_clean_model.head()

,COD_CICLO,COD_MATERIAL,COD_CANAL,COD_REGIAO,qt_venda_liquida,PCT_DESCONTO,VL_PRECO,FLG_CAMPANHA_RESUMO,pct_devolucao,DES_CATEGORIA_MATERIAL,...,price_lag_6,y_lag_12,discount_lag_12,price_lag_12,y_roll_mean_3,y_roll_mean_6,y_roll_std_6,y_roll_cv_6,y_rel_vs_roll_6,ciclo_str
0,201802,100692,anon_S0,anon_S1,18.0,0.0,521.4,0,0.928571,anon_S12,...,NaN,NaN,NaN,NaN,6.0,6.0,NaN,NaN,3.000000,02
1,201803,100692,anon_S0,anon_S1,6.0,0.0,653.4,0,0.975000,anon_S12,...,NaN,NaN,NaN,NaN,12.0,12.0,8.485281,0.707107,0.500000,03
2,201807,100692,anon_S0,anon_S1,6.0,0.0,653.4,0,0.975000,anon_S12,...,NaN,NaN,NaN,NaN,10.0,10.0,6.928203,0.692820,0.600000,07
3,201813,100692,anon_S0,anon_S1,0.0,0.0,485.4,0,1.000000,anon_S12,...,NaN,NaN,NaN,NaN,10.0,9.0,6.000000,0.666667,0.000000,13
4,201901,100692,anon_S0,anon_S1,6.0,0.0,485.4,0,0.975000,anon_S12,...,NaN,NaN,NaN,NaN,4.0,7.2,6.572671,0.912871,0.833333,01
